In [1]:
import matplotlib.image as mpimg
import numpy as np
import matplotlib.pyplot as plt
import os,sys
from scipy import ndimage
from PIL import Image
from sklearn.cluster import KMeans
from sklearn import linear_model
from sklearn import svm
from sklearn import preprocessing as prp
from helpers_img_my import *
from Post_processing import *
#from helpers_img import *
from sklearn import model_selection as modsel
from preprocessing_my import *

In [49]:
def post_processing(label,threshold,size_min,verbarg,horbarg,size_image):
    label = complete_lines(label,threshold)
    label = remove_isolated_connected_component(label,size_min)
    label = clean_garbage_vert(label,verbarg,size_image)
    label = clean_garbage_hor(label,horbarg,size_image)
    label = remove_isolated_connected_component(label,size_min)
    #label = complete_semilines(label,0.85, size_image)
    return label

In [ ]:
# Settings
foreground_threshold = 0.25 # percentage of pixels > 1 required to assign a foreground label to a patch
patch_size = 16 # each patch is 16*16 pixels
deg=2

root_dir = "training/"
image_dir = root_dir + "images/"
gt_dir = root_dir + "groundtruth/"
files = os.listdir(image_dir)

#n=len(files)
n=4
k_fold=2 #number of folds in the k-fold cross-validation

imgs = [load_image(image_dir + files[i]) for i in range(n)]
gt_imgs = [load_image(gt_dir + files[i]) for i in range(n)]

seed = 1
np.random.seed(seed)

kf=modsel.KFold(n_splits=k_fold,shuffle=True)
kf.get_n_splits(imgs)
lambdas=np.logspace(-8,-5,3) # 1/lambda
thresh=np.linspace(0.35,0.55,3)
shape=(lambdas.shape[0],thresh.shape[0])
mean_f1=np.zeros(shape)

for id_lam,lam in enumerate(lambdas):
    for id_t,t in enumerate(thresh):
        nb_f1_te = np.zeros(k_fold)
        for ind,[train_id,test_id] in enumerate(kf.split(imgs)):
            im_tr=[imgs[idx] for idx in train_id]
            im_te=[imgs[idx] for idx in test_id]
            gt_tr=[gt_imgs[idx] for idx in train_id]
            gt_te=[gt_imgs[idx] for idx in test_id]

            im_tr,gt_tr = rotation(im_tr,gt_tr)
            im_tr,gt_tr = flip(im_tr,gt_tr)
            im_tr=[add_features(im_tr[i]) for i in range(len(im_tr))]
            im_te=[add_features(im_te[i]) for i in range(len(im_te))]


            im_tr = [img_crop(im_tr[i], patch_size, patch_size) for i in range(len(im_tr))]
            im_te = [img_crop(im_te[i], patch_size, patch_size) for i in range(len(im_te))]
            gt_tr = [img_crop(gt_tr[i], patch_size, patch_size) for i in range(len(gt_tr))]
            gt_te = [img_crop(gt_te[i], patch_size, patch_size) for i in range(len(gt_te))]

            im_tr = np.asarray([im_tr[i][j] 
                              for i in range(len(im_tr)) 
                              for j in range(len(im_tr[i]))])
            im_te = np.asarray([im_te[i][j] 
                              for i in range(len(im_te)) 
                              for j in range(len(im_te[i]))])
            gt_tr =  np.asarray([gt_tr[i][j] 
                              for i in range(len(gt_tr)) 
                              for j in range(len(gt_tr[i]))])
            gt_te =  np.asarray([gt_te[i][j] 
                              for i in range(len(gt_te)) 
                              for j in range(len(gt_te[i]))])

            im_patches_tr=[extract_features(im_patches_tr[i]) for i in range(len(im_tr))]
            im_patches_te=[extract_features(im_patches_te[i]) for i in range(len(im_te))]

            im_tr = np.asarray([poly_features(im_tr[i],deg) for i in range(len(im_tr))])
            gt_tr = np.asarray([value_to_class(np.mean(gt_tr[i])) for i in range(len(gt_tr))])

            im_te = np.asarray([poly_features(im_te[i],deg) for i in range(len(im_te))])
            gt_te = np.asarray([value_to_class(np.mean(gt_te[i])) for i in range(len(gt_te))])

            ridgereg = linear_model.Ridge(alpha=lam, normalize=True)
            ridgereg.fit(im_tr, gt_tr)

            Z_te = ridgereg.predict(im_te)
            Z_te=1*(Z_te>t)

            Z_pp=[]
            for i in range(len(gt_te)):
                Z_pp = Z_pp + post_processing(Z_te[i*625:(i+1)*625],18,9,3,3,25)

            nb_f1_te[ind]=compute_F1(gt_te, Z_pp)
        
        mean_f1[id_lam,id_t]=nb_f1_te.mean()
    
best=np.max(mean_f1)
ids=np.where(mean_f1==best)
best_lambda=lambdas[ids[0][0]]
best_thresh=thresh[ids[1][0]]

In [47]:
best

0.71569148434997265

In [48]:
mean_f1

array([[ 0.66933381,  0.67223512,  0.5618237 ,  0.41963531,  0.3253012 ],
       [ 0.71569148,  0.68196733,  0.53480782,  0.36928817,  0.31299105],
       [ 0.70834749,  0.67044995,  0.446406  ,  0.32625995,  0.24066024]])

In [16]:
thres=np.linspace(0.3,0.8,11)

In [22]:
zers=np.zeros((thres.shape[0],lambdas.shape[0]))
np.argmax(zers)

0

In [18]:
thres.shape[0]

11

In [27]:
matrix=np.asarray([[1,2,3],[4,5,6]])
matrix

array([[1, 2, 3],
       [4, 5, 6]])

In [34]:
ids=np.where(matrix==3)
ids[1][0]

2